In [ ]:
# This notebook: consists of acquiring data from server by months
#                and exporting to separate .csv files..
#                Contains functions for creating shifts and differences and basic control
#                Contains basic controll functions for time interval consistency and invalid values counts   

In [1]:
#@title Imports
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from datetime import datetime
import glob

In [ ]:
# If Python version is 3.9.13 you can use pyarrow
# you can remove printing lines in order to make the code faster

monthly_li =  ["%02d" % i for i in range(6,12)] #creates list of months in '01' format
for mon in monthly_li:
    print(f"Filtering {mon}")
    t1 = datetime.now()
    path = f"/uufs/chpc.utah.edu/common/home/u0035056/uu2dvar/csv/2022{mon}*.csv"
    filenames = glob.glob(path)
    df_month = pd.DataFrame()
    for filename in filenames:
        df= pd.read_csv(filename, delimiter=',',index_col=0,header=0,skiprows=[1])
        df1 = df[(df.Latitude<41.34) & (df.Latitude > 40.17) & (df.Longitude < -111.346) & (df.Longitude > -112.678)]
        df_month=pd.concat([df_month,df1])
    t2 = datetime.now()
    took = t2 - t1
    print(f"it took {took} seconds to parse {mon} month.")
    df_month.to_csv(f'./df_{mon}.csv')

In [ ]:
# Compiled into UDF within coordinates window

def get_dat_for_coordinates(months_start,months_end,lat1 = 41.34,lat2 = 40.17 ,lon1 = -111.346,lon2 = -112.678):
    outp_df = pd.DataFrame()
	monthly_li =  ["%02d" % i for i in range(months_start,months_end+1)] # generates list of months in string format ’01’, ’02’…
	for mon in monthly_li:
    		print(f"Filtering {mon}")
    		t1 = datetime.now()
    		path = f"/uufs/chpc.utah.edu/common/home/u0035056/uu2dvar/csv/2022{mon}*.csv"
    		filenames = glob.glob(path)
    		df_wbb_mon = pd.DataFrame()
    	for filename in filenames:
                df= pd.read_csv(filename, delimiter=',',index_col=0,header=0,skiprows=[1])
        		df1 = df[(df.Latitude< lat1) & (df.Latitude > lat2) & (df.Longitude < lon1) & (df.Longitude > lon2)]
        		df_wbb_mon=pd.concat([df_wbb_mon,df1])
    	t2 = datetime.now()
    	took = t2 - t1
    	print(f"it took {took} seconds to parse {mon} month.")
    	df_wbb_mon.to_csv(f'./df_{mon}.csv')
        outp_df=pd.concat([outp_df,df_wbb_mon])
return outp_df

In [ ]:
# Read data for one station only from 2022

filenames = glob.glob("/uufs/chpc.utah.edu/common/home/u0035056/uu2dvar/csv/2022*.csv")
df_wbb = pd.DataFrame()
for filename in filenames:
    df= pd.read_csv(filename, delimiter=',',index_col=0,header=0,skiprows=[1])
    df1 = df[df.Station=='WBB']
    df_wbb=pd.concat([df_wbb,df1])

In [ ]:
# Functions creating differences between rows in  periods defined by user
# Call them after data cleaning - differences could be 0 otherwise and it could bias dataset

import warnings
warnings.filterwarnings("ignore")

def create_shifts(df,periods,column_name_li):
  #Creates shifts in periods for dataframe and names i.e. what is value in previous one or two terms?
    for name_li in column_name_li:
        df[str(name_li) + "sh_" + str(periods)] = df[name_li].shift(periods = periods)
    return df

def create_differences(df,periods,column_name_li):
    # Yields a lot of warnings - figure it out
    for name_li in column_name_li:
        df[str(name_li) + "dif_" + str(periods)] = df[name_li].diff(periods = periods)
    return df

In [3]:
# these list concatenations create lists of features that could be interesting
# in the last line, you can slice your dataset by desired series

feat_li = ['yo_', 'd_', 'xb_', 'xa_', 'stdev_']
feat2_li = ['t','td','u','v','ws','rh'] 

#df_wbb2 = df_wbb2.loc[df_wbb2[tes_val] != -9999]

nw_li = [st+ap for ap in feat2_li for st in feat_li ]
feat_to_plot = [('d_'+str(j)) for j in ['t','td','u','v','ws','rh']]
#df1 = df_wbb2[feat_to_plot]

In [ ]:
# Data control and time intervall
# Two phase funciton for dropping values of -9999 and those 
# where interval is bigger than usual (majority of measurements)

In [58]:
# control dates and measurement intervals
# returns dataframe with station, percentage of diferent measurement intervals 
# and number of consistent measurement times

def inspect_time_regularity(df)
    import warnings
    warnings.filterwarnings("ignore")

    stations, percentage, consistent = [],[],[] # three metrics of measurement periodicity
    for stat in df['Station'].unique():
        subset = df.loc[df['Station'] == stat]
        time_ser = df['(Index) -> Time']
        subset['Time'] = subset['(Index) -> Time'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M %Z") )
        #df['Time_diff'] = pd.to_timedelta(time_ser.astype(str)).diff(1).dt.total_seconds().div(60)
        subset['datetime_d'] = subset['Time'].diff(1).astype('timedelta64[h]')
        count = len(subset['datetime_d'].value_counts()) # number of unique values in difference
        try:
            a = list(subset['datetime_d'].value_counts())
            a_m, a_s = max(a), sum(a)
            perc = (a_s - a_m)/a_s
            print(f'Station {stat} contains {perc} fraction of irregular measurement times.')
        
        except ValueError:
            print(f'Station {stat} yielded error.')
            perc = 100
            a_m = 0
        stations.append(stat)
        percentage.append(perc)
        consistent.append(a_m)

    freq_df = pd.DataFrame()
    freq_df['Station'] = stations # station name
    freq_df['Percantatge'] = percentage # % of irregular measurements - different that majority
    freq_df['Consistent'] = consistent # no. of majority intervals
    return freq_df

Station COOPHOLU1 yielded error.


In [73]:
def inspect_column(df,col):
    stations, nans, nines = [],[],[]
    Navals_df = pd.DataFrame()
    for stat in df['Station'].unique():
        subset = df.loc[df['Station'] == stat]
        count_na = subset['err_xa_ws'].isna().sum()
        count_99 = subset.loc[subset[col] == -9999.0].shape[0]
        stations.append(stat)
        nans.append(count_na)
        nines.append(count_99)
    
    Navals_df['Stations'] = stations # station name
    Navals_df['NaNs_'+str(col)] = nans # % of irregular measurements - different that majority
    Navals_df['Nin_'+str(col)] = nines # no. of majority intervals
    return Navals_df

u = inspect_column(df,'d_ws')

,Stations,NaNs_d_ws,Nin_d_ws
0,WBB,0,0
1,HOL,0,0
2,SNI,0,671
3,SBE,0,3
4,SB2,0,4
